In [1]:
import pandas as pd
import datetime as dt
import json
import glob
import os

In [2]:
match_id_list = ['3869685', '3869519', '3857289', '3869117', '3857278', '3857272', '3857270', '3857262']

In [5]:
for match in match_id_list:
    fname =  "open-data/data/events/" + match + ".json"
    df = pd.read_json(fname)
    
    # Extract values from .json objects
    df['event_type'] = [event['id'] for event in df['type']]
    df['event_name'] = [event['name'] for event in df['type']]
    df['play_pattern'] = [play['name'] for play in df['play_pattern']]
    df['possession_team'] = [team['name'] for team in df['possession_team']]
    df['team'] = [team['name'] for team in df['team']]
    
    # Remove unnecessary columns
    df.drop(['index','type'], inplace=True, axis=1)
    
    team1, team2 = set(df['team'])
    
    # Create shots dataframe
    shots_df = df[df['event_name']=="Shot"]
    shots_df = shots_df[['period', 'timestamp', 'minute', 
                         'second', 'possession_team', 'play_pattern', 
                         'team', 'duration', 'shot']]
    
    # Extract values form shots objects
    shots_df['xg'] = [shot['statsbomb_xg'] for shot in shots_df['shot']]
    shots_df['outcome'] = [shot['outcome'] for shot in shots_df['shot']]
    shots_df['outcome'] = [shot['name'] for shot in shots_df['outcome']]
    shots_df = shots_df.reset_index().drop(['index', 'shot'], axis=1)
    
    # Create new features
    shots_df['event_type'] = 'SHOT'
    shots_df['offset_in'] = dt.timedelta(seconds=1)
    shots_df['offset_out'] = dt.timedelta(seconds=5)
    shots_df['event_in_mp'] = pd.Series(dtype='datetime64[ns]')
    shots_df['event_out_mp'] = pd.Series(dtype='datetime64[ns]')
    
    # Save shots_df as excel file for each match
    shots_df.to_excel(f'shots/{match}_{team1}_{team2}_shots.xlsx')